In [1]:
import numpy as np
import os
import pandas as pd
import SimpleFunctions as SF
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]


NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

# Folder for the results
Optim_Path = os.path.join('OptimizationResults', BCsType, ConstitutiveModel)

OptiName = str(NumberElements) + 'Elements.txt'
MarcName = str(NumberElements) + 'Elements-M.txt'

# Cost Function Data
CostData = pd.read_csv('CostFunctionGrid/Results.csv', sep=',', decimal='.')
CostData = CostData.dropna()
OptiData = pd.read_csv(os.path.join(Optim_Path, OptiName), sep=' ', decimal='.')
MarcData = pd.read_csv(os.path.join(Optim_Path, MarcName), sep=' ', decimal='.')

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCostNormMax'].idxmin()])

# Nus    = CostData.Nu
# Mus    = CostData.Mu
# Alphas = CostData.Alpha
# Costs  = CostData.TotalCostNormMax

# DM, MuM, AlphaM = 1.026, 0.458, -59.367
# NuM = (3-DM*MuM)/(DM*MuM+6)
# Marcdf = pd.DataFrame([NuM, MuM, AlphaM])

# Threshold = 1E-0
# Subsampling = 1

# Vmin = Costs.min()
# Vmax = Threshold * (Costs.max() - Costs.min())

# # 3D Plot
# %matplotlib widget
# plt.rc('figure', figsize=[12,7])
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# CostImage = ax.scatter(Nus[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs[0::Subsampling], cmap=plt.gray(), marker = '.', vmin=Vmin, vmax=Vmax, label='Points tested')
# ColorBar=plt.colorbar(CostImage)
# ColorBar.set_label('Cost (-)')
# ax.set_xlabel('Nu (-)')
# ax.set_ylabel('Mu (kPa)')
# ax.set_zlabel('Alpha (-)')
# ax.plot([Nus[CostData['TotalCostNormMax'].idxmin()]], [Mus[CostData['TotalCostNormMax'].idxmin()]], [Alphas[CostData['TotalCostNormMax'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
# ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m', label='Budday starting point')
# ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m', label='Optimization path')
# ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
# ax.plot([MarcData.Nu[0]], [MarcData.Mu[0]], [MarcData.Alpha[0]], marker = 'o', color='c', label='Marc starting point')
# ax.plot(MarcData.Nu, MarcData.Mu, MarcData.Alpha, marker = None, color='c', label='Optimization path')
# ax.plot([MarcData.Nu[len(MarcData)-1]], [MarcData.Mu[len(MarcData)-1]], [MarcData.Alpha[len(MarcData)-1]], marker = 'o', color='b', label='Optimization result')
# ax.plot(Marcdf.loc[0], Marcdf.loc[1], Marcdf.loc[2], marker = 'o', color='g', label='Marc optimization result')
# ax.legend()
# plt.show()

Min cost point for:
Alpha                             -24
BCsType                         Fixed
CompressionCostNoNorm       0.0296841
CompressionCostNormMax      0.0531391
CompressionCostStep           9.18372
ConstitutiveModel               Ogden
Mu                                0.6
Nu                               0.41
NumberElements                     20
SimpleShearCostNoNorm      0.00813879
SimpleShearCostNormMax       0.116729
SimpleShearCostStep            2.2651
TensileCostNoNorm         0.000807557
TensileCostNormMax           0.032357
TensileCostStep               1196.05
TotalCostNoNorm             0.0128768
TotalCostNormMax            0.0674084
TotalCostStep                   402.5
Name: 34778, dtype: object


In [4]:
# Filtering

Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]

FilteredRangeValues = [0,0.1]

MaxRow = CostData[Color].idxmax()

MaxCost = CostData[Color].max()

FilteredCost = CostData.dropna()
        
for Point in FilteredCost[Color].index:
    CostValue = FilteredCost[Color][Point]
    if CostValue < FilteredRangeValues[0] or CostValue > FilteredRangeValues[1] :
        FilteredCost = FilteredCost.drop([Point])
        

In [5]:
# Filtered 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = FilteredCost.Nu
Mus    = FilteredCost.Mu
Alphas = FilteredCost.Alpha
Costs  = FilteredCost[Color]

# img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[FilteredCost[Color].idxmin()]], [Mus[FilteredCost[Color].idxmin()]], [Alphas[FilteredCost[Color].idxmin()]], marker = 'o', color='k')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r')
ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m')
ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r')
ax.plot([MarcData.Nu[0]], [MarcData.Mu[0]], [MarcData.Alpha[0]], marker = 'o', color='c')
ax.plot(MarcData.Nu, MarcData.Mu, MarcData.Alpha, marker = None, color='c')
ax.plot([MarcData.Nu[len(MarcData)-1]], [MarcData.Mu[len(MarcData)-1]], [MarcData.Alpha[len(MarcData)-1]], marker = 'o', color='b')
ax.plot(Marcdf.loc[0], Marcdf.loc[1], Marcdf.loc[2], marker = 'o', color='g')
# Build the legend
ax.plot([],[], marker = 'o', linestyle='none', color='m', label='Budday starting point')
ax.plot([],[], color='m', label='Optimization path')
ax.plot([],[], marker = 'o', linestyle='none', color='r', label='Optimization result')
ax.plot([],[], marker = 'o', linestyle='none', color='c', label='Marc starting point')
ax.plot([],[], color='c', label='Optimization path')
ax.plot([],[], marker = 'o', linestyle='none', color='b', label='Optimization result')
ax.plot([],[], marker = 'o', linestyle='none', color='g', label='Marc optimization result')
ax.plot([],[], marker = '.', linestyle='none', color='y', label='Points tested')
ax.plot([],[], marker = 'o', linestyle='none', color='k', label='Grid min cost point')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
%matplotlib widget

def f(x, y):
    plt.clf()
    plt.plot(x, y, marker='o')
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    fig.canvas.draw_idle()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
interactive_plot = interactive(f, x=(-10.0, 10.0), y=(-10.0, 10.0))
interactive_plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='x', max=10.0, min=-10.0), FloatSlider(value=0.0, des…

In [ ]:
%matplotlib widget

# Plot data
Axes  = ['Nu', 'Mu', 'Alpha']
Units = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]
NContours = 10

Indices = [0,1,2,0,1,2]

plt.rc('figure', figsize=[8,12])
fig = plt.figure()

MinPoint = CostData[Color].idxmin()
    
for Projection in range(len(Axes)):

    # Plot tuning
    Xaxis     = Axes[Indices[Projection]]
    Xunits    = Units[Indices[Projection]]
    Yaxis     = Axes[Indices[Projection+1]]
    Yunits    = Units[Indices[Projection+1]]
    SliceAxis = Axes[Indices[Projection+2]]

    # Generate data for contour
    Slice4Contour = [CostData[SliceAxis][MinPoint]]
    Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color]]
    Slice = Slice.drop_duplicates([Yaxis,Xaxis])

    Levels = Slice.sort_values(by=[Color])[Color].to_numpy()

    i     = 1
    Pos   = int(i*len(Levels)/(NContours+1))
    ContourSelection = np.zeros(len(Levels))

    while Pos < len(Levels):
        ContourSelection[Pos] = 1
        i  += 1
        Pos = int(i*len(Levels)/(NContours+1))


    SampledLevels = ContourSelection*Levels
    SampledLevels = SampledLevels[SampledLevels != 0]
    
    SampledLevels = np.insert(SampledLevels, 0, 0)
    SampledLevels = np.append(SampledLevels,Levels[-1])

    Table = Slice.pivot(Yaxis,Xaxis,Color)

    X = Table.columns.values
    Y = Table.index.values
    Z = Table.values

    XY,YX = np.meshgrid(X, Y)
    
    X = np.tile(X,Z.shape[0])
    Y = np.repeat(Y,Z.shape[1])
    ZZ = np.reshape(Z,(1,Z.shape[0]*Z.shape[1]))[0]
    ZZ = np.array(ZZ)
#     ZZ[~np.isnan(ZZ)] = 1
    
    GridPoints = pd.DataFrame({'X':X,'Y':Y,'Z':ZZ})
    
    Weights = np.linspace(len(Levels)-1,1,len(Levels)-1)
    Slopes = []
    for Index in range(len(Levels)-1):
        Slope = Levels[Index+1]-Levels[Index]
        Slopes.append(Slope*Weights[Index])

    NewColorScale = np.linspace(1,len(Levels),len(Levels)) * np.mean(Slopes) / len(Levels)

    Vmin = 0
    Vmax = max(NewColorScale)

    # Plot
    ax = fig.add_subplot(3,1,Projection+1)
    GridPoints.plot(kind='scatter',x='X',y='Y',c='Z', marker = '.', ax=ax, colorbar=False, cmap='viridis', vmin=Vmin, vmax=Vmax)
    plt.contour(XY, YX, Z, vmin=Vmin, vmax=Vmax, levels=SampledLevels, colors='black', linewidths=0.5)
    CostContour = plt.contourf(XY, YX, Z, vmin=Vmin, vmax=Vmax, levels=SampledLevels, cmap='viridis', alpha=0.85)
    ColorBar=plt.colorbar(CostContour)
    ColorBar.set_label('Cost (-)')
    plt.clim(Vmin,Vmax)
    plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
    plt.plot(OptiData[Xaxis][len(OptiData)-1], OptiData[Yaxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    ax.set_xlabel(Xaxis + Xunits)
    ax.set_ylabel(Yaxis + Yunits)
    if Projection+1 == 3:
        plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
        plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData[Color].min()))
        plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
        plt.legend(loc='lower left', framealpha=1)
    plt.subplots_adjust(left=0.1, bottom=0.05, right=0.975, top=0.95, wspace=None, hspace=None)
    plt.show()